# COSC 4368 Group Project :'(

Important Information for Jackson's implementation of Agent Class and environment space:
* Gamestate is a list of integers, [3,3,5,3,1,3,0,0,0,0,0,0,5,5,5]. 
* Legal Moves is a list of Chars, ["P", "D", "N", "E", "S", "W"]

Below will be the agent class that will create objects with a Q Table and an ID. 

It currently has three non-declaration functions 
1. getLegalMoves(): takes in gamestate and using the ID of the agent derives all legal moves for the agent. returns a list of capital letter chars
2. updateGamestate(): takes in gamestate and nextmove and applies the next move to the game state. returns a list of integers representative of the environment
3. step(): calls previous methods to produce the whole step of the agent given the game state. returns a list of integers representative of the environment and should update the agent's Qtable when appropriate

Todo 
* implement the Q-Table
* implement functions relating to the Q-Table, update, build, etc
* Next Move according to policy needs to be implemented

In [1]:
import numpy as np
import random

In [12]:
class Agent:
    # this class is used to define the different agents

    def __init__(self, agentID):
        # self.qtable = None            To be filled out later
        self.agentID = agentID
    
    def getLegalMoves(self, gamestate):
        legalMoves = ["P", "D", "N", "S", "E", "W"]
        positions = gamestate[0:6]

        ipositions = [positions[0], positions[2], positions[4]]
        agentI = ipositions.pop(self.agentID)

        jpositions = [positions[1], positions[3],  positions[5]]
        agentJ = jpositions.pop(self.agentID)

        otherAgentPositions = [[ipositions[0], jpositions[0]], [ipositions[1], jpositions[1]]]

        agentBox = gamestate[self.agentID+6]

        dropoff = gamestate[9:12]
        dropoffPositions = [[1,1], [3,1], [4,5]]

        pickup = gamestate[12:]
        pickupPositions = [[1,5], [2,4], [5,2]]

        if(agentBox == 0):
            legalMoves.remove("D")
            if([agentI, agentJ] not in pickupPositions):
                legalMoves.remove("P")
            else:
                index = 0
                for location in pickupPositions:
                    if ([agentI, agentJ] == location and pickup[index] == 0):
                        legalMoves.remove("P")
                    index += 1
        else:
            legalMoves.remove("P")
            if([agentI, agentJ] not in dropoffPositions):
                legalMoves.remove("D")
            else:
                index = 0
                for location in dropoffPositions:
                    if ([agentI, agentJ] == location and dropoff[index] == 5):
                        legalMoves.remove("D")
                    index += 1

        if([agentI-1, agentJ] in otherAgentPositions or agentI-1 == 0):
            legalMoves.remove("N")
        if([agentI+1, agentJ] in otherAgentPositions or agentI+1 == 6):
            legalMoves.remove("S")
        if([agentI, agentJ-1] in otherAgentPositions or agentJ-1 == 0):
            legalMoves.remove("W")   
        if([agentI, agentJ+1] in otherAgentPositions or agentJ+1 == 6):
            legalMoves.remove("E")   

        return legalMoves

    def updateGameState(self, gamestate, nextMove):
        agentI = gamestate[self.agentID*2]
        agentJ = gamestate[self.agentID*2+1]

        dropoffPositions = [[1,1], [3,1], [4,5]]
        pickupPositions = [[1,5], [2,4], [5,2]]

        if(nextMove == "D"):
            index = 0
            for location in dropoffPositions:
                if([agentI, agentJ] == location):
                    gamestate[9 + index] = gamestate[9 + index] + 1
                index += 1
            gamestate[6+self.agentID] = 0

        if(nextMove == "P"):
            index = 0
            for location in pickupPositions:
                if([agentI, agentJ] == location):
                    gamestate[12 + index] = gamestate[12 + index] - 1
                index += 1
            gamestate[6+self.agentID] = 1

        if(nextMove == "N"):
            gamestate[self.agentID*2] = agentI-1
        if(nextMove == "S"):
            gamestate[self.agentID*2] = agentI+1
        if(nextMove == "W"):
            gamestate[self.agentID*2+1] = agentI-1
        if(nextMove == "E"):
            gamestate[self.agentID*2+1] = agentI+1

        return gamestate
    
    # def step(self, gamestate):
    #     # Method representing one step of the agent in the game
    #     legalmoves = self.getLegalMoves(gamestate)
    #     nextMove = None                                   # will be changed when Q Table is implemented
    #     # updateQTable(nextMove)
    #     newGameState = self.updateGameState(gamestate, nextMove)
    #     return newGameState

    def step(self, gamestate, q_table, policy):
        # Get legal moves for the agent
        legal_moves = self.getLegalMoves(gamestate)
        
        # Select action based on the Q-table and policy
        if "P" in legal_moves:
            next_move = "P"
        elif "D" in legal_moves:
            next_move = "D"
        else:
            # If neither pickup nor dropoff is available, use Q-table-based policy
            next_move = q_table.select_action((gamestate[self.agentID*2], gamestate[self.agentID*2+1]), policy, legal_moves)
        
        # Update game state based on the selected move
        new_game_state = self.updateGameState(gamestate, next_move)
        
        return new_game_state

        

In [13]:
redAgent = Agent(0)
blueAgent = Agent(1)
blackAgent = Agent(2)

initialEnvironment = [3,3,5,3,1,3,0,0,0,0,0,0,5,5,5]


The Q-table class uses a constructor that initializes the Q-table with zeros based on the size of the environment and learning parameters. The function get_q_value retrieves the Q-value for a given state-action pair, while update_q_value updates it using the Bellman equation considering the immediate reward and the next state. The print_q_table function displays the Q-table in a readable format, organizing Q-values for each state-action pair. The select_action function chooses an action according to specified policies: randomly from legal moves, with a probability-based exploitation of Q-values, or greedily exploiting Q-values. Finally, the exploit_action function selects an action by exploiting the Q-values, preferring the one with the highest Q-value for the state, randomly breaking ties.

Parameters definitions within Q-Table functions:

* state: A tuple representing the current state (coordinates) of the agent in the environment.
* action: An integer representing the action taken by the agent.
* reward: The immediate reward received by the agent for taking an action.
* next_state: A tuple representing the next state (coordinates) of the agent after taking an action.
* policy: A string representing the policy used for action selection.
* legal_moves: A list of strings representing the legal moves available to the agent from the current state.

In [14]:
class QTable:
    def __init__(self, environment_rows, environment_cols, num_actions, learning_rate, discount_factor):
        # Initialize QTable with environment parameters and zero-initialized Q-values
        self.environment_rows = environment_rows
        self.environment_cols = environment_cols
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_table = np.zeros((environment_rows, environment_cols, num_actions))

    def get_q_value(self, state, action):
        # Retrieve Q-value for a given state-action pair
        return self.q_table[state[0], state[1], action]

    def update_q_value(self, state, action, reward, next_state):
        # Update Q-value for a given state-action pair using the Bellman equation
        current_q_value = self.q_table[state[0], state[1], action]
        max_next_q_value = np.max(self.q_table[next_state[0], next_state[1]])
        new_q_value = (1 - self.learning_rate) * current_q_value + self.learning_rate * (reward + self.discount_factor * max_next_q_value)
        self.q_table[state[0], state[1], action] = new_q_value

    def print_q_table(self):
        # Print the Q-table in a readable format
        print("Q-table:")
        print("State    |   N       E       S       W")
        print("----------------------------------------")
        for row in range(self.environment_rows):
            for col in range(self.environment_cols):
                print(f"({row},{col})   | ", end="")
                for action_idx in range(self.num_actions):
                    q_value_str = f"{self.q_table[row, col, action_idx]:.2f}"  
                    padding = max(0, 7 - len(q_value_str))
                    print(" " * (padding // 2) + q_value_str + " " * ((padding + 1) // 2), end="")
                print()

    def select_action(self, state, policy, legal_moves):
        # Select an action based on the specified policy
        if policy == "PRANDOM":
            return random.choice(legal_moves)
        elif policy == "PEXPLOIT":
            # With probability 0.8, exploit the Q-values; otherwise, choose a random action
            if random.random() < 0.8:
                return self.exploit_action(state, legal_moves)
            else:
                return random.choice(legal_moves)
        elif policy == "PGREEDY":
            # Greedily select the action with the highest Q-value
            return self.exploit_action(state, legal_moves)

    def exploit_action(self, state, legal_moves):
        # Exploit the Q-values by selecting the action with the highest Q-value for the given state
        max_q_value = np.max(self.q_table[state[0], state[1]])
        max_indices = np.argwhere(self.q_table[state[0], state[1]] == max_q_value).flatten()
        chosen_action_index = random.choice(max_indices)
        return legal_moves[chosen_action_index]
